## Bibliotecas e módulos

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import csv
import numpy as np

import sklearn.preprocessing as preproc
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import mixture

#from matplotlib import rcParams
#rcParams['text.usetex'] = True
#rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

## Função para leitura dos dados

In [ ]:
def read_class_data(path):
    with open(path, newline='') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:
            try:
                data = np.vstack( (data , np.asarray(row).astype(np.float) ) )
            except:
                data = np.asarray(row).astype(np.float)
    f.close()
    y = data[:,0]
    x = data[:,1:]
    return y,x

## Funções utilizadas pelo método FKM

In [ ]:
#critério inicialização dos lambdas-----------------------------
def compute_lambdas(data,mu,beta):
    expo = 2.0/(beta-1.0)
    m,_ = data.shape
    k,_ = mu.shape
    lamb = np.zeros((m,k))
    diss = np.zeros((k))
    for i in range(0,m):
        for j in range(0,k):
            diss[j] = np.linalg.norm( data[i,:] - mu[j,:] )+0.0001
        
        for j in range(0,k):
            s = 0
            for ell in range(0,k):
                s += ( diss[j] / diss[ell] )**expo
            lamb[i,j] = 1/s
    return lamb


#Atualização dos centroides
def compute_mu(data,lamb,beta):
    m,dim = data.shape
    _,k = lamb.shape
    mu = np.zeros((k,dim))
    for j in range(0,k):
        num = np.zeros((dim))
        den = 0
        for i in range(0,m):
            num[:] += (lamb[i,j]**beta) * data[i,:]
            den += (lamb[i,j]**beta)
        mu[j,:] = num[:]/den
    return mu


#critério inicialização----------------------------------------
def init_centroids(data,k):
    m,dim = data.shape
    bestCost = 10**10
    mu = np.zeros((k,dim)) #inicializacao dos 'mus' [usando matriz]
    bestMu = np.zeros((k,dim))
    for _ in range(0, int(np.ceil(0.1*m)) ):
        randPos = np.random.randint(0,m,k)
        mu = data[randPos,:]
        cost = intra_cluster_var(data,mu)
        if cost < bestCost:
            bestCost = cost
            bestMu = np.copy(mu)
    return bestMu


#critério inicialização----------------------------------------
def intra_cluster_var(data,mu):
    m,_ = data.shape
    k,_ = mu.shape
    
    totalCost = np.zeros((k))
    countClus = np.zeros((k))
    for i in range(0,m):
        diss = np.zeros((k))
        for j in range(0,k):
            diss[j] = np.linalg.norm( data[i,:] - mu[j,:] )**2
        pos = np.argmin(diss)
        totalCost[pos] += diss[pos]
        countClus[pos] += 1
    clusterVar = (1/m) * np.sum(totalCost)

    return clusterVar


#Função para definir o agrupamento segundo o lambda---
def assign_label(lamb):
    m,_ = lamb.shape
    lab = np.zeros((m))
    for i in range(0,m): lab[i] = np.argmax(lamb[i,:])
    return lab


#Método Fuzzy K-Means---------------------------------
def fuzzy_kmeans(data,k,beta,epsilon):
    #Inicialização do mu e lambda
    mu = init_centroids(data,k)
    lamb = compute_lambdas(data,mu,beta)

    while True:  #inicio do processo iterativo ("while True para simular um laço do-while")
        mu_old = np.copy(mu) #Para fins de comparação/convergênca
        mu = compute_mu(data,lamb,beta) #Cálculo dos centroides
        lamb = compute_lambdas(data,mu,beta) #Atualização dos lambdas
        
        if np.linalg.norm( mu - mu_old ) < epsilon: break #Teste de convergência
        
    #Definição dos agrupamentos segundo as pertinencias
    lab = assign_label(lamb)
    return lab

## Conjunto de dados (World Air Quality Index):
* Dados sobre qualiade do ar - Air Quality Open Data Platform - Worldwide COVID-19 dataset
* Informações adicionais: https://aqicn.org/data-platform/covid19/
* Os dados coletados são os mais próximos do dia 15/março/2020
* Variáveis:
    * Latitude
    * Longitude
    * wind-speed ($m/s$)
    * wind-gust ($m/s$)
    * humidity ($\%$)
    * dew ($C$)
    * pressure ($mb$ - millibars)
    * precipitation (inch $1.0 inch \approx 25.4 mm$)
    * temperature ($C$)
    * CO - monóxido de carbono ($\mu g/m^3 \rightarrow$ EPA Standard)
    * pm10 - particulas respiráveis ($\mu g/m^3 \rightarrow$ EPA Standard)
    * pm25 - particulas finas ($\mu g/m^3 \rightarrow$ EPA Standard)
    * SO2 - dióxido de enxofre ($\mu g/m^3 \rightarrow$ EPA Standard)
    * NO2 - dióxido de nitrogênio ($\mu g/m^3 \rightarrow$ EPA Standard)
    * O3 - ozônio ($\mu g/m^3 \rightarrow$ EPA Standard)

* Detalhes sobre as unidades: https://weather.gladstonefamily.net/CWOP_Guide.pdf
* EPA Standard: https://en.wikipedia.org/wiki/National_Ambient_Air_Quality_Standards

In [ ]:
path = 'waqi.csv'

#Não existe rótulo (y) válido para este conjunto de dados
_,xOrig = read_class_data(path) 

#Escolha dos atributos
Atributos = [2,3,4,5,6,7,8,9,10,11,12,13,14]

#Normalização dos dados...
x = np.copy(xOrig[:,Atributos])
x = preproc.normalize(x, norm='max', axis=0, copy=True)



In [ ]:
path = 'MapWAQI.png'
FS = (15,5) #Tamanho da figura a ser gerada
fig = plt.figure(constrained_layout=True,figsize=FS)
spec = gridspec.GridSpec(ncols=1, nrows=1, figure=fig)

ax = fig.add_subplot(spec[0, 0])

ax.scatter( xOrig[:,1], xOrig[:,0], c='blue', alpha=0.5)
ax.set_xlabel('Longitude',fontsize=20)
ax.set_ylabel('Latitude',fontsize=20)

ax.set_aspect('equal', 'box')

ax.legend(fontsize=10)
ax.grid(True)

plt.savefig(path,dpi=300)

## Parametrização e execução

In [ ]:
path_out = 'saida_mude_este_nome.png'
n_clusters = 4 #Usar até 8 ou acrescentar mais cores no vetor 'cores' 
epsilon = 0.0001
beta = 2.0
#----------

#===Instanciação e execução de agrupamento hierárquico
#aglom = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward', affinity='euclidean')
#res = aglom.fit(x)
#agrup = res.labels_

#===Instanciação e execução do algoritmo K-Médias
km = KMeans(n_clusters=n_clusters, max_iter=1000, tol=epsilon)
agrup = km.fit_predict(x)

#===Execução dos algoritmo FKM
#agrup = fuzzy_kmeans(x,n_clusters,beta,epsilon)

#===Instanciação e modelagem via GMM
#gmm = mixture.GaussianMixture(n_components=n_clusters)
#gmm.fit(x)
#agrup = []
#for i in range(x.shape[0]): agrup.append( np.argmax( gmm.predict_proba( [x[i,:]] ) ) )
#agrup = np.array(agrup)

## Visualização do resultado de agrupamento

In [ ]:
FS = (15,5) #Tamanho da figura a ser gerada
fig = plt.figure(constrained_layout=True,figsize=FS)
spec = gridspec.GridSpec(ncols=1, nrows=1, figure=fig)

cores = ['red','blue','green','orange','magenta','cyan','darkred','grey']
marcadores = ['o','+','^','*','D','x','p','h']


ax = fig.add_subplot(spec[0,0])
for i in range(n_clusters):
    pos = np.where( agrup == i )
    ax.scatter(xOrig[pos, 1], xOrig[pos, 0], marker=marcadores[i], c=cores[i], s=50)
    
ax.set_xlabel('Longitude',fontsize=20)
ax.set_ylabel('Latitude',fontsize=20)
ax.set_aspect('equal', 'box')
ax.legend(fontsize=10)
ax.grid(True) 

plt.savefig(path_out,dpi=300)
plt.show()